<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/EP_week_2_LC_and_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# JP Morgan, Bank of America, HSBC, Citigroup

In [ ]:
!unzip transcripts.zip

Archive:  transcripts.zip
   creating: transcripts/
  inflating: __MACOSX/._transcripts  
  inflating: transcripts/.DS_Store   
  inflating: __MACOSX/transcripts/._.DS_Store  
  inflating: transcripts/1q23-earnings-transcript.pdf  
  inflating: __MACOSX/transcripts/._1q23-earnings-transcript.pdf  
  inflating: transcripts/201103-3q20-equity-analysts-meeting-transcript.pdf  
  inflating: __MACOSX/transcripts/._201103-3q20-equity-analysts-meeting-transcript.pdf  
  inflating: transcripts/2024-04-16 BAC 1Q Earnings Call_ADA.pdf  
  inflating: __MACOSX/transcripts/._2024-04-16 BAC 1Q Earnings Call_ADA.pdf  
  inflating: transcripts/230908-interim-results-2023-fixed-income-investors-conference-call-transcript.pdf  
  inflating: __MACOSX/transcripts/._230908-interim-results-2023-fixed-income-investors-conference-call-transcript.pdf  
  inflating: transcripts/jpm-2q24-earnings-call-transcript-final.pdf  
  inflating: __MACOSX/transcripts/._jpm-2q24-earnings-call-transcript-final.pdf  
  infla

**Install packages**

- `langchain`: The core framework to build the RAG pipeline. It manages the interactions between Phi (the language model) and retrieval systems, handling tasks like prompt management, memory, chaining, and tool integration.

- `chromadb`: A vector database for storing and retrieving embeddings (vector representations of text). This enables the retrieval component in RAG, where relevant documents or knowledge snippets are stored and retrieved based on similarity to the user query.

- `pypdf`: A library to parse and extract text from PDF files. Useful for ingesting content into the vector database, allowing the RAG system to process knowledge stored in PDF documents.

- `openai`: The OpenAI API client library, used if Phi or other models from OpenAI are accessed. It manages API calls to OpenAI’s servers.

- `sentence-transformers`: A library to generate embeddings for text. Often used in the retrieval step to convert queries and documents into vectors, enabling similarity searches in chromadb.

- `accelerate`: Helps optimize the performance of language models by enabling distributed training and efficient hardware utilization, especially useful when running models like Phi locally or on specific hardware.

- `langchain-community`: Adds community-contributed tools, integrations, and utilities to LangChain, which can include additional retrieval or model support.

In [ ]:
!pip install langchain chromadb pypdf openai sentence-transformers accelerate langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.5 MB/s eta 0:0

# Phi

Ignore this, I used Smollm instead

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True,)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
llm = HuggingFacePipeline(pipeline=pipe)

# RAG

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import Document
import os
import torch

In [ ]:
# Load Embeddings
embedding_model = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# Load SmolLM Model & Tokenizer
model_name = "HuggingFaceTB/SmolLM-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Set Up LLM Pipeline with Better Generation Settings
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.5,
    top_p=0.8,
    repetition_penalty=1.3,
    do_sample=True
)
llm = HuggingFacePipeline(pipeline=pipe)

# Load PDFs
folder_path = "/content/transcripts/"
file_names = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]

documents = []
for pdf_path in file_names:
    try:
        loader = PyPDFLoader(os.path.join(folder_path, pdf_path), extract_images=False)
        documents.extend(loader.load_and_split())
        print(f"Loaded: {pdf_path}")
    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")

# Chunking Strategy for Better Retrieval
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

# Store in ChromaDB
db = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="test_index")
db.persist()

# Load Vector Database & Retriever
vectordb = Chroma(persist_directory="test_index", embedding_function=embeddings)
#retriever = vectordb.as_retriever(search_kwargs={"k": 5})
retriever = vectordb.as_retriever(search_kwargs={"k": 3})  # ✅ Reduce retrieval size


# Conversation Memory (Stores past user inputs)
#memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory = ConversationBufferWindowMemory(k=3, memory_key="chat_history", return_messages=True)

def reset_memory_if_needed():
    """Clears memory after 3 exchanges to prevent excessive token growth."""
    if len(memory.chat_memory.messages) > 6:  # Each exchange has a user+bot message
        print("\n🛑 **Memory Full: Resetting Conversation History...**\n")
        memory.clear()


system_prompt = """You are analyzing a bank's quarterly earnings call transcript.
Extract and summarize key financial insights, avoiding unnecessary details.
If the answer isn't found, respond with "I don't know."
Provide sources for your answers at the end.

Your response format:
- **Answer:** [Concise response]
- **Key Insights:** [Bullet-pointed highlights]
- **Sources:** [List of document sources used]

<|user|>
Context:
{context}

Question: {question}<|end|>
<|assistant|>"""

# Clean User Input
def clean_user_input(user_input):
    return user_input.strip().replace("\n", " ").replace("\t", " ")

# Remove Duplicate Chunks
def remove_duplicate_chunks(chunks):
    seen = set()
    unique_chunks = []
    for chunk in chunks:
        chunk_text = chunk.page_content.strip()
        if chunk_text not in seen:
            seen.add(chunk_text)
            unique_chunks.append(chunk)
    return unique_chunks

# Truncate Retrieved Context to Avoid Model Overload ... still happens TODO: Need to refine
def truncate_context(context_list, max_tokens=1000):
    """Trims retrieved context while keeping document metadata, based on token count."""
    truncated_docs = []
    current_tokens = 0

    for doc in context_list:
        doc_tokens = len(tokenizer.encode(doc.page_content))

        if current_tokens + doc_tokens <= max_tokens:
            truncated_docs.append(doc)
            current_tokens += doc_tokens
        else:
            break  # Stop adding when token limit is reached

    return truncated_docs

#
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=retriever, memory=memory, verbose=False
)


<ipython-input-2-2606d276c26f>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-2-2606d276c26f>:21: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


Loaded: 3Q24-Earnings-Transcript.pdf
Loaded: 2024q1tr.pdf
Loaded: jpm-2q24-earnings-call-transcript-final.pdf
Loaded: 4Q23-Earnings-Transcript.pdf
Loaded: 230908-interim-results-2023-fixed-income-investors-conference-call-transcript.pdf
Loaded: 1q23-earnings-transcript.pdf
Loaded: 3Q23-Earnings-Transcript.pdf
Loaded: tr200714a.pdf
Loaded: tr201013a.pdf
Loaded: transcript07122024.pdf
Loaded: 240801-interim-results-2024-presentation-to-investors-and-analysts-transcript.pdf
Loaded: 2024-01-12 BAC 4Q Earnings Call.pdf
Loaded: 2q23-earnings-transcript.pdf
Loaded: 201103-3q20-equity-analysts-meeting-transcript.pdf
Loaded: jpm-4q22-earnings-call-final-transcript.pdf
Loaded: 2024-04-16 BAC 1Q Earnings Call_ADA.pdf
Loaded: jpm-3q24-earnings-call-transcript-final.pdf
Loaded: jpm-3Q22-earnings-call-final-transcript.pdf
Loaded: tr200415a.pdf
Loaded: 240502-1q-2024-hsbc-presentation-to-investors-and-analysts-transcript.pdf
Loaded: 2023-10-17 BAC 3Q Earnings Call.pdf
Loaded: 2Q23 Earnings Transcript

<ipython-input-2-2606d276c26f>:42: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
<ipython-input-2-2606d276c26f>:45: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="test_index", embedding_function=embeddings)
<ipython-input-2-2606d276c26f>:52: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=3, memory_key="chat_history", return_messages=True)


In [ ]:
def format_response(question, response):
    """Formats the chatbot response to only return the question and answer."""

    # Ensure the response is clean
    response_text = response.strip()

    # If model outputs instructions or extra text, remove it
    unwanted_phrases = [
        "Use the following pieces of context",
        "If you don't know the answer, just say that you don't know",
        "Don't try to make up an answer."
    ]

    for phrase in unwanted_phrases:
        if phrase in response_text:
            response_text = response_text.split(phrase)[-1].strip()  # Keep only the last part

    # Ensure the output is structured as needed
    formatted_output = f"Question: {question}\nHelpful Answer: {response_text}"

    return formatted_output

def trim_final_input(question, context, max_tokens=1024):
    """Ensures final model input does not exceed token limit."""
    system_message = """You are analyzing a bank's quarterly earnings call transcript.
    Extract and summarize key financial insights, avoiding unnecessary details.
    If the answer isn't found, respond with "I don't know."
    Provide sources for your answers at the end."""

    input_text = f"{system_message}\n\nContext:\n{context}\n\nQuestion: {question}"

    tokens = tokenizer.encode(input_text, truncation=True, max_length=max_tokens)

    return tokenizer.decode(tokens)




In [ ]:
print("\n💬 **Bank Earnings Chatbot** (Type 'exit' to stop)")

while True:
    user_input = input("\n🔹 **You:** ")
    if user_input.lower() == "exit":
        print("\n**Exiting Chatbot. Have a great day!**")
        break

    user_input = clean_user_input(user_input)

    reset_memory_if_needed()

    context = retriever.get_relevant_documents(user_input)
    context = remove_duplicate_chunks(context)
    context = truncate_context(context, max_tokens=800)

    print("\n **Retrieved Context:**")
    for doc in context:
        print(f"- **Source:** {doc.metadata.get('source', 'Unknown Source')}, Page {doc.metadata.get('page', 'Unknown Page')}")

    formatted_input = trim_final_input(user_input, context, max_tokens=1024)

    response = qa_chain({"question": formatted_input})

    print(f"\n{format_response(user_input, response['answer'])}")


💬 **Bank Earnings Chatbot** (Type 'exit' to stop)

🔹 **You:** What is the current position of most banks on investments in q3 2024


<ipython-input-4-90b2a7fb032f>:13: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(user_input)
<ipython-input-4-90b2a7fb032f>:23: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": formatted_input})



 **Retrieved Context:**
- **Source:** /content/transcripts/jpm-4q22-earnings-call-final-transcript.pdf, Page 13
- **Source:** /content/transcripts/2023-10-17 BAC 3Q Earnings Call.pdf, Page 11
- **Source:** /content/transcripts/2023-10-17 BAC 3Q Earnings Call.pdf, Page 4

Question: What is the current position of most banks on investments in q3 2024
Helpful Answer: , don't try to make up an answer.

Fourth Quarter 2023 Earnings Announcement 
January 12, 2024 
2 
 
a pretentious aura as the economists predicted a mild recession within the year. Instead, 2023 showcased 
economic resilience led by U.S. consumers, despite higher interest rates. 
We ended 2023 with economists projecting the Fed has successfully steered the U.S. economy to a soft 
landing. In regards to the economy, during 2023, we consistently made a few points regarding what we 
were seeing in our customer data here at Bank of America. First, the year-over-year growth rate and 
spending from the beginning of '23 started de

KeyboardInterrupt: Interrupted by user